## Import

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [66]:
data = [pd.read_csv('apply_train.csv'),
pd.read_csv('company.csv'),
pd.read_csv('recruitment.csv'),
pd.read_csv('resume_certificate.csv'),
pd.read_csv('resume_education.csv'),
pd.read_csv('resume_language.csv'),
pd.read_csv('resume.csv'),
# pd.read_csv('sample_submission.csv'),
]



In [62]:
df3[0]

,recruitment_seq,company_type_seq,supply_kind,employee
0,R02073,2,514,20
1,R03274,2,402,90
2,R02195,2,514,20
3,R03372,4,100,60
4,R00867,2,402,590
...,...,...,...,...
2372,R01786,2,100,100
2373,R03415,2,100,270
2374,R04028,4,402,525
2375,R06508,2,402,70


In [68]:
# df1 = pd.DataFrame('apply_train.csv')
df1 = pd.read_csv('apply_train.csv'),


### recruit 관련
df2 = pd.read_csv('company.csv'),
df3 = pd.read_csv('recruitment.csv'),


## resume 관련 
df4 = pd.read_csv('resume_certificate.csv'),
df5 = pd.read_csv('resume_education.csv'),
df6 = pd.read_csv('resume_language.csv'),
df7 = pd.read_csv('resume.csv'),

# df8 = pd.read_csv('sample_submission.csv'),

for df in data:
    print(df.columns)
    
    



Index(['resume_seq', 'recruitment_seq'], dtype='object')
Index(['recruitment_seq', 'company_type_seq', 'supply_kind', 'employee'], dtype='object')
Index(['recruitment_seq', 'address_seq1', 'address_seq2', 'address_seq3',
       'career_end', 'career_start', 'check_box_keyword', 'education',
       'major_task', 'qualifications', 'text_keyword'],
      dtype='object')
Index(['resume_seq', 'certificate_contents'], dtype='object')
Index(['resume_seq', 'hischool_type_seq', 'hischool_special_type',
       'hischool_nation', 'hischool_gender', 'hischool_location_seq',
       'univ_type_seq1', 'univ_type_seq2', 'univ_transfer', 'univ_location',
       'univ_major', 'univ_sub_major', 'univ_major_type', 'univ_score'],
      dtype='object')
Index(['resume_seq', 'language', 'exam_name', 'score'], dtype='object')
Index(['resume_seq', 'reg_date', 'updated_date', 'degree', 'graduate_date',
       'hope_salary', 'last_salary', 'text_keyword', 'job_code_seq1',
       'job_code_seq2', 'job_code_seq3', 

In [69]:
import csv

mydict = {}
with open("apply_train.csv", "r") as file:
    reader = csv.reader(file)
    attrs = []
    result = []
    for row in reader:
        if len(attrs) < 1:
            attrs = row
            continue
        temp_row_dict = dict()
        for key, value in zip(attrs, row):
            temp_row_dict[key] = value
        result.append(temp_row_dict)


for row in result:
    print(row)
    break

{'\ufeffresume_seq': 'U05833', 'recruitment_seq': 'R03838'}


## Data Load

In [4]:
apply_train_df = pd.read_csv('apply_train.csv')
apply_train_df

,resume_seq,recruitment_seq
0,U05833,R03838
1,U06456,R02144
2,U07807,R01877
3,U04842,R02463
4,U08336,R00112
...,...,...
57941,U02270,R03430
57942,U02640,R04987
57943,U08238,R01342
57944,U01296,R06363


In [5]:
my_2darray = np.array([[1, 2, 3], [4, 5, 6]])
display(pd.DataFrame(my_2darray))

# Take a dictionary as input to your DataFrame 
my_dict = {"a": ['1', '3'], "b": ['1', '2'], "c": ['2', '4']}
display(pd.DataFrame(my_dict))

,0,1,2
0,1,2,3
1,4,5,6


,a,b,c
0,1,1,2
1,3,2,4


## User-Item Matrix / Similarity / Score

In [104]:
user = list(apply_train_df['resume_seq'].unique())
company = list(apply_train_df['recruitment_seq'].unique())


In [6]:
# 사용자-아이템 행렬 생성: 구직자가 해당 채용 공고에 지원했으면 1, 아니면 0으로 설정
user_item_matrix = apply_train_df.groupby(['resume_seq', 'recruitment_seq']).size().unstack(fill_value=0)
aaa = np.array(user_item_matrix)
print(sum(sum(aaa)))
user_item_matrix[user_item_matrix > 1] = 1

print(user_item_matrix.shape)
# 사용자 간의 유사성 계산
user_similarity = cosine_similarity(user_item_matrix)

# 추천 점수 계산
user_predicted_scores = user_similarity.dot(user_item_matrix) / np.array([np.abs(user_similarity).sum(axis=1)]).T

(8482, 6695)


In [143]:
np.abs(user_similarity).sum(axis=0).shape

#user similarity 는 symmetric

(8482,)

In [128]:
aa = user_similarity.dot(user_item_matrix)
aa.shape

(8482, 6695)

In [127]:
bb =np.array([np.abs(user_similarity).sum(axis=1)]).T 
bb.shape

(8482, 1)

## Prediction

In [7]:
# 이미 지원한 채용 공고 제외하고 추천
recommendations = []
for idx, user in enumerate(user_item_matrix.index):
    # 해당 사용자가 지원한 채용 공고
    applied_jobs = set(user_item_matrix.loc[user][user_item_matrix.loc[user] == 1].index)
    
    # 해당 사용자의 추천 점수 (높은 점수부터 정렬)
    sorted_job_indices = user_predicted_scores[idx].argsort()[::-1]
    recommended_jobs = [job for job in user_item_matrix.columns[sorted_job_indices] if job not in applied_jobs][:5]
    
    for job in recommended_jobs:
        recommendations.append([user, job])

## Submission

In [8]:
    # sample_submission.csv 형태로 DataFrame 생성
top_recommendations = pd.DataFrame(recommendations, columns=['resume_seq', 'recruitment_seq'])

top_recommendations.to_csv('./baseline_submit.csv', index=False)